# О том, как собирался корпус

При создании корпуса я подбирала предложения из НКРЯ и интернета, в которых есть грамматическая омонимия: совпадение форм существительного и глагола (*ели*, *печь*), причастия и прилагательного (*следующий*), наречия и прилагательного, наречия и производного предлога (*навстречу*), предлога и деепричастия (*благодаря*), числительного и глагола (*три*), существительного и прилагательного (*жаркое*) и т.д. Кроме того, я добавила имена собственные, которые могут быть похожи на существительное или прилагательное (на -ов, -их. Чтобы правильно разметить часть речи в подобных сложных случаях, парсер должен ориентироваться на контекст и синтаксическую структуру предложения.

Для разметки я использовала немного измененный набор тэгов Universal Dependencies, поскольку он наиболее распространенный и отличается умеренной детализацией тэгов: все предлоги я обозначала одним тэгом CONJ, отказалась от тэга AUX, однако причастия и деепричастия обозначала тэгами PRTF и GRND соответственно, как это сделано в OpenCorpora, так как мне было необходимо понять, сможет ли парсер отличить эти части речи от, например, прилагательных. Полный тэгсет выглядит так:  

VERB - глагол  
NOUN - существительное  
ADJ - прилагательное  
PRTF - причастие  
GRND - деепричастие  
PRON - местоимение  
DET - притяжательное местоимение  
NUM - числительное  
PART - частица  
INTJ - междометие  
CONJ - союз  
ADP - предлог  
ADV - наречие  
PROPN - имя собственное 

# Тэггинг

Я использовала для сравнения pymorphy2, natasha и spacy.

In [1]:
!pip install pymorphy2

You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install natasha

You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install spacy --upgrade

You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 15.3/15.3 MB 8.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [5]:
import pymorphy2
import spacy
import nltk
nltk.download("punkt")
nlp = spacy.load("ru_core_news_sm")
morph = pymorphy2.MorphAnalyzer()

from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, Doc, MorphVocab

segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Прогоняем корпус через тэггеры, сохраняем результат в виде списка кортежей. Для spacy и natasha я также сохраняла грамемму, обозначающую форму глагола, чтобы потом при приведении разметки к универсальным тэгам можно было глаголы заменить на деепричастия и причастия. 

In [6]:
corpus = """Наш основной разыгрывающий Вадим Хамуцких повредил палец и отправился на скамейку запасных.
Штирлиц шел по лесу и увидел голубые ели.
Что кинул он в краю родном?
Что ж, это классика, господа! И сколько бы режиссеры сериалов не пытались «переплюнуть» Рязанова, у них это не получится.
― Что ты всё думаешь о себе? ― спрашивает жена.
Между блюдом майонеза и жарким ставятся по две кучки тарелок для жаркого.
В Эстонии ― эстонский для русских, русский для эстонцев.
Всемогущий Горшков обещал «устроить» ее на прямой поезд, следующий через станцию Семеновка, а там и до поселка рукой подать: всего двадцать пять километров.
А прыгать туда нельзя, ща поезд следующий проедет, вон расписание.
Поздно вечером подруги собирались у невесты для завивки ей волос, чтобы подготовить причёску к венцу
Буду сидеть, курить, наслаждаться весенним утром, свежим воздухом и молодою пахучею зеленью недавно распустившихся деревьев
Жарко в небе солнце летнее.
Я плакала и жарко молилась.
А я иду к тебе навстречу, и я несу тебе цветы.
Все полевые цветы тянутся навстречу солнцу.
Она помолилась и теперь, бла­годаря Бога за избавление, вернулась к себе.
Благодаря отцу я и сестры знаем французский, немецкий и английский языки.
Три дня не переставая шел дождь.
Хорошенько три паркет щеткой!
Пила раза два выпрыгнула, меняя место, словно ей было неулёжно, потом въелась и пошла.
Год назад я не пила кофе после обеда.
Печь нужна, чтобы печь в ней пирожки."""

In [7]:
doc_spacy = nlp(corpus)
spacy_pos = []
for token in doc_spacy:
  spacy_pos.append((token, token.pos_, token.morph.get('VerbForm')))

In [8]:
pymorphy_pos = []
for word in nltk.word_tokenize(corpus):
  pymorphy_pos.append((word, morph.parse(word)[0].tag.POS))

In [9]:
natasha_pos = []
doc_natasha = Doc(corpus)
doc_natasha.segment(segmenter)
doc_natasha.tag_morph(morph_tagger)
for token in doc_natasha.tokens:
  natasha_pos.append((token.text, token.pos, token.feats))

Эта функция приводит тэгсет spacy и natasha к формату ручной разметки: в случае, если парсер разметил часть речи как глагол, смотрим на граммему 'VerbForm'.

In [10]:
def ud_tags_conventer(pair, parser):
    word = pair[0]
    tag = pair[1]
    dict = {"AUX": "VERB", "SCONJ": "CONJ", "CCONJ": "CONJ", 'Conv' : 'GRND', 'Part' : 'PRTF', 'Fin' : 'VERB', 'Inf' : 'VERB'}
    form = ""
    if tag in dict.keys():
        new_tag = dict[tag]
    else:
        if tag == 'VERB':
            if parser == 'natasha':
                form = pair[2]['VerbForm']
                new_tag = dict[form]
            if parser == 'spacy':
                form = pair[2][0]
                new_tag = dict[form]
        else:
            new_tag = tag
    return((word, new_tag))


Эта функция приводит тэгсет pymorphy2 к формату ручной разметки: здесь важно было все притяжательные местоимения заменить на DET, так как они размечались как краткие прилагательные (для этого смотрим, есть ли тэг Apro в разметке)

In [11]:
def pymorphy_converter(pair):
    word = pair[0]
    tag = pair[1]
    tags = {
        "ADJF": "ADJ",
        "ADJS": "ADJ",
        "COMP": "ADJ",
        "INFN": "VERB",
        "NUMR": "NUM",
        "ADVB": "ADV",
        "NPRO": "PRON",
        "PRED": "ADV",
        "PREP": "ADP",
        "PRCL": "PART",
        "PRTS": "PRTF", 
        None : "PUNCT"
    }
    if 'Apro' in morph.parse(word)[0].tag:
        new_tag = 'DET'
    else:
        if tag in tags.keys():
            new_tag = tags[tag]
        else:
            new_tag = tag
    return((word, new_tag))

Приводим к единому формату:

In [12]:
pymorphy_pos_norm = []
natasha_pos_norm = []
spacy_pos_norm = []
for el in pymorphy_pos:
    pymorphy_pos_norm.append(pymorphy_converter(el))
for el in spacy_pos:
    if not el[1] == 'SPACE':
        spacy_pos_norm.append(ud_tags_conventer(el, 'spacy'))

In [13]:
for el in natasha_pos:
    natasha_pos_norm.append(ud_tags_conventer(el, 'natasha'))

In [14]:
import pandas as pd

In [15]:
df_natasha = pd.DataFrame(natasha_pos_norm, columns=['Word', 'POS'])
df_spacy = pd.DataFrame(spacy_pos_norm, columns=['Word', 'POS'])
df_pymorphy = pd.DataFrame(pymorphy_pos_norm, columns=['Word', 'POS'])

In [16]:
base_df = pd.read_csv("corpus_pos.txt", sep="\t")

In [17]:
from sklearn.metrics import accuracy_score

## Считаем accuracy

In [18]:
def count_accuracy(parser_df, base):
    parser_tags = list(parser_df['POS'])
    base_tags = list(base['POS'])
    print("Accuracy: %.4f" % accuracy_score(parser_tags, base_tags))


In [19]:
count_accuracy(df_pymorphy, base_df)

Accuracy: 0.8869


In [20]:
count_accuracy(df_natasha, base_df)

Accuracy: 0.8942


In [21]:
count_accuracy(df_spacy, base_df)

Accuracy: 0.8978


Лучшим тэггером для русского оказался spacy. 

В предыдущем домашнем задании я работала с текстами на английском, но для этой домашки в первой части не решилась составлять корпус на английском, поскольку я не носитель, не знаю тонкостей английской морфологии и не смогла бы верно разметить части речи. Тем не менее, во второй части для написания чанкера буду использовать spacy - предполагаю, что для английского он бы тоже показал лучший результат.

## Создаем chunker

Буду выделять следующие группы: 
- not + ADJ -- это позволит "поймать" случаи типа not good, когда отрицание меняет тональность на противоположную
- ADV + ADJ -- такие сочетания ярче передают эмоциональную окраску, так как наречие усиливает значение прилагательного
- not + VERB -- это случаи типа (did) not like, которые тоже влияют на тональность

In [22]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 12.8/12.8 MB 7.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [23]:
from nltk.corpus import stopwords
stops = stopwords.words('english')
stops.remove('not')
nlp_eng = spacy.load("en_core_web_sm")

In [24]:
print(stops)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', '

изменим функцию lemmatize: вместо nltk будем использовать spacy, а еще добавим параметр 'pos', чтобы можно было в результате препроцессинга получить пары лемма - часть речи

In [25]:
def lemmatize(x, pos = True, returnList = True):
    if type(x) != str:
        return ""
    tokens = nlp_eng(x.lower())
    result = []
    for word in tokens:
        if word.text.isalpha():
            nf = word.lemma_
            w_pos = word.pos_
            if nf not in stops:
                if pos == True:
                    result.append((nf, w_pos))
                else:
                    result.append(nf)
    if returnList == True:
        return(result)
    else:
        return " ".join(result)

In [46]:
def get_bigrams(tokens):
    bigrams = nltk.bigrams(tokens)
    res = []
    for el in bigrams:
        first = el[0]
        second = el[1]
        if first[1] == 'ADV' and second[1] == 'ADJ':
            res.append(first[0] + ' ' + second[0])
        if first[0] == 'not' and second[1] == 'ADJ':
            res.append(first[0] + ' ' + second[0])
        if first[0] == 'not' and second[1] == 'VERB':
            res.append(first[0] + ' ' + second[0])
    return res


код из предыдущей домашки:

In [27]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
session = requests.session()

In [45]:
def parse_page(url):
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    return soup

In [29]:
def get_albums(url): #функция, которая определяет список альбомов исполнителя
    albums = []
    page = parse_page(url)
    r = page.find_all('td', {'class' : 'title brief_metascore'})
    for album in r:
        l = album.find('a').attrs['href']
        albums.append(f'https://www.metacritic.com{l}/user-reviews')
    return albums

In [30]:
k_albums = get_albums('https://www.metacritic.com/person/kanye-west')

In [31]:
def parse_reviews(url): #функция, которая парсит рецензии, на вход подается ссылка на страницу с альбомом
    all_reviews = []
    page = parse_page(url)
    reviews = page.find_all('div', {'class' : 'review_content'})
    for review in reviews:
        dic = {}
        dic['text'] = review.find_all('div', {'class' : 'review_body'})[0].text.strip()
        dic['grade'] = review.find_all('div', {'class' : 'review_grade'})[0].text.strip()
        all_reviews.append(dic)
    return all_reviews

In [32]:
reviews = []
for album in k_albums:
    reviews.extend(parse_reviews(album))

In [33]:
import pandas as pd
data = pd.DataFrame(reviews) #превращаем данные в датафрейм

In [34]:
def grade_to_sentiment(x): #выделяем положительные и отрицательные отзывы
    if x > 10: #некоторые оценки на сайте по 100-балльной шкале
        sent = x / 10
    else:
        sent = x
    if sent <= 5:
        return 0
    else:
        return 1

In [35]:
data['grade'] = data['grade'].astype(float)  
data['sentiment'] = data['grade'].apply(grade_to_sentiment)
data

,text,grade,sentiment
0,Immensely mediocre. I hope Ye doesn't make any...,3.0,0
1,very poor album with poor production and a lot...,3.0,0
2,It's clearly unfinished. The best is yet to co...,7.0,1
3,"This review contains spoilers, click expand to...",7.0,1
4,"Donda, Donda, DondaDonda, Donda, Donda, Donda,...",10.0,1
...,...,...,...
1183,Now I usually like the rock music. Actually no...,9.0,1
1184,This album was the BOMB!,10.0,1
1185,Like every hip-hop album (even the great ones)...,70.0,1
1186,Most producers who approach the mic do so at t...,70.0,1


In [36]:
data['lemmas'] = data['text'].apply(lemmatize, args=(False, True))
data['pos'] = data['text'].apply(lemmatize, args=(True, True))

In [37]:
data

,text,grade,sentiment,lemmas,pos
0,Immensely mediocre. I hope Ye doesn't make any...,3.0,0,"[immensely, mediocre, I, hope, ye, make, album...","[(immensely, ADV), (mediocre, ADJ), (I, PRON),..."
1,very poor album with poor production and a lot...,3.0,0,"[poor, album, poor, production, lot, throwaway...","[(poor, ADJ), (album, NOUN), (poor, ADJ), (pro..."
2,It's clearly unfinished. The best is yet to co...,7.0,1,"[clearly, unfinished, good, yet, come, far, ba...","[(clearly, ADV), (unfinished, ADJ), (good, ADJ..."
3,"This review contains spoilers, click expand to...",7.0,1,"[review, contain, spoiler, click, expand, view...","[(review, NOUN), (contain, VERB), (spoiler, NO..."
4,"Donda, Donda, DondaDonda, Donda, Donda, Donda,...",10.0,1,"[donda, donda, dondadonda, donda, donda, donda...","[(donda, PROPN), (donda, PROPN), (dondadonda, ..."
...,...,...,...,...,...
1183,Now I usually like the rock music. Actually no...,9.0,1,"[I, usually, like, rock, music, actually, I, u...","[(I, PRON), (usually, ADV), (like, VERB), (roc..."
1184,This album was the BOMB!,10.0,1,"[album, bomb]","[(album, NOUN), (bomb, NOUN)]"
1185,Like every hip-hop album (even the great ones)...,70.0,1,"[like, every, hip, hop, album, even, great, on...","[(like, ADP), (every, DET), (hip, NOUN), (hop,..."
1186,Most producers who approach the mic do so at t...,70.0,1,"[producer, approach, mic, peril, dropout, west...","[(producer, NOUN), (approach, VERB), (mic, ADJ..."


In [38]:
from collections import Counter

In [39]:
def collect_freqlist(reviews, reviews_pos): #создаем частотные списки, делаем из них множества
    freqlist = Counter()
    all_bigrams = []
    for text in reviews:
        for word in text:
            if word.isalpha():
                freqlist[word] += 1
    for text in reviews_pos:
        bigrams = get_bigrams(text)
        all_bigrams.extend(bigrams)
    l = [*dict(freqlist).keys()]
    l.extend(all_bigrams)
    freqlist_set = set(l)
    return freqlist_set

In [40]:
positive = data[data['sentiment'] == 1]['lemmas'].to_list()
positive_with_pos = data[data['sentiment'] == 1]['pos'].to_list()
negative = data[data['sentiment'] == 0]['lemmas'].to_list()
negative_with_pos = data[data['sentiment'] == 0]['pos'].to_list()

In [47]:
pos_set = collect_freqlist(positive, positive_with_pos)
neg_set = collect_freqlist(positive, negative_with_pos)
pos_only = pos_set.difference(neg_set) #выделяем множества только позитивных и негативных слов
neg_only = neg_set.difference(pos_set)

In [42]:
def predict_sentiment(review): #простейшая функция для определения тональности отзыва
    prep_review = lemmatize(review, False, True)
    pos_points = 0
    neg_points = 0
    bigr = list(nltk.bigrams(prep_review))
    for word in prep_review:
        if word in pos_only:
            pos_points += 1
        if word in neg_only:
            neg_points += 1
    for el in bigr:
        b = el[0] + ' ' + el[1]
        if b in pos_only:
            pos_points += 1
        if b in neg_only:
            neg_points += 1
    if pos_points > neg_points:
        return 1
    else:
        return 0


///в прошлой дз я зачем-то сделала бесполезный шаг с делением на test/train, но не успела его переделать его для этой домашки, поэтому пришлось оставить

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data[['text', 'lemmas']], data[['sentiment']], test_size=0.2, random_state=42)

print(len(X_train), 'training reviews')
print(len(X_test), 'testing reviews')

950 training reviews
238 testing reviews


In [44]:
from sklearn.metrics import accuracy_score

y_train_pred = X_train['text'].apply(predict_sentiment)
print('train accuracy:', accuracy_score(y_train, y_train_pred))

train accuracy: 0.5747368421052632


точность предсказания понизилась :( скорее всего, я что-то сделала не так...